In [1]:
!python -V

Python 3.9.18


In [2]:
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import pandas as pd
import xgboost as xgb

In [3]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error

In [4]:
import mlflow

mlflow.set_tracking_uri("sqlite:///../mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/mlruns/1', creation_time=1702972736474, experiment_id='1', last_update_time=1702972736474, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [5]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    # Calculate duration
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    # Filter between 1 min and 60 mins
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']    
    df[categorical] = df[categorical].astype(str)
    
    return df

In [6]:
df_train = read_dataframe("../data/green_tripdata_2021-01.parquet")
df_val = read_dataframe("../data/green_tripdata_2021-02.parquet")

In [7]:
len(df_train), len(df_val)

(73908, 61921)

**Note:** There was an issue in applying separate transforms i.e. X_train and X_val yielded different shapes. So I applied transform after combining them and then separated them again. 

In [8]:
df_combined = pd.concat([df_train, df_val], axis=0)

In [9]:
df_combined['PU_DO'] = df_combined['PULocationID'] + '_' + df_combined['DOLocationID']

In [10]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()
combined_dicts = df_combined[categorical + numerical].to_dict(orient='records')
X_combined = dv.fit_transform(combined_dicts)

In [11]:
X_train = X_combined[:73908, :]
X_val = X_combined[73908:, :]

In [12]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
# Prepare data for xgboost
train = xgb.DMatrix(X_train, label=y_train)
val = xgb.DMatrix(X_val, label=y_val)

Train xgboost model.

In [16]:
with mlflow.start_run():

    # Use best params to train model
    params = {
        "learning_rate": 0.20472169880371677,
        "max_depth": 17,
        "min_child_weight": 1.2402611720043835,
        "objective": "reg:linear",
        "reg_alpha": 0.28567896734700793,
        "reg_lambda": 0.004264404814393109,
        "seed": 42
    }

    mlflow.log_params(params)

    booster = xgb.train(
                params=params,
                dtrain=train,
                num_boost_round=1000,
                evals=[(val, "validation")],
                early_stopping_rounds=50
            )
    
    y_pred = booster.predict(val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("../models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("../models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="model")

/home/pytholic/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [07:57:28] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:10.62105
[1]	validation-rmse:9.45454
[2]	validation-rmse:8.61244
[3]	validation-rmse:8.02161
[4]	validation-rmse:7.60613
[5]	validation-rmse:7.31430
[6]	validation-rmse:7.11506
[7]	validation-rmse:6.97578
[8]	validation-rmse:6.87711
[9]	validation-rmse:6.80659
[10]	validation-rmse:6.75482
[11]	validation-rmse:6.71870
[12]	validation-rmse:6.68975
[13]	validation-rmse:6.67024
[14]	validation-rmse:6.65272
[15]	validation-rmse:6.63910
[16]	validation-rmse:6.62823
[17]	validation-rmse:6.62004
[18]	validation-rmse:6.61336
[19]	validation-rmse:6.60866
[20]	validation-rmse:6.60602
[21]	validation-rmse:6.60323
[22]	validation-rmse:6.60192
[23]	validation-rmse:6.60053
[24]	validation-rmse:6.59835
[25]	validation-rmse:6.59609
[26]	validation-rmse:6.59387
[27]	validation-rmse:6.59214
[28]	validation-rmse:6.59088
[29]	validation-rmse:6.58932
[30]	validation-rmse:6.58731
[31]	validation-rmse:6.58478
[32]	validation-rmse:6.58333
[33]	validation-rmse:6.58150
[34]	validation-rmse:6.

/home/pytholic/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [07:59:01] WARNING: /workspace/src/c_api/c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


Make predictions.

In [16]:
logged_model = 'runs:/f3e39da461024416b0b3a9de30b5b2c7/model'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

/home/pytholic/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [04:57:31] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [17]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: model
  flavor: mlflow.xgboost
  run_id: f3e39da461024416b0b3a9de30b5b2c7

In [18]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

/home/pytholic/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/xgboost/core.py:160: UserWarning: [04:57:33] WARNING: /workspace/src/objective/regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


In [19]:
xgboost_model

In [20]:
y_pred = xgboost_model.predict(val)
y_pred[:10]

array([14.809331,  7.256571, 15.71568 , 24.437902,  9.105063, 17.193743,
       11.460967,  8.388062,  9.049488, 18.53353 ], dtype=float32)

Train different models.

In [13]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (LinearRegression, Lasso, Ridge, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "../data/green_tripdata_2021-01.parquet")
        mlflow.log_param("val-data-path", "../data/green_tripdata_2021-02.parquet")
        mlflow.log_artifact("../models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

2023/12/21 04:59:04 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2023/12/21 04:59:07 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/pytholic/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."
2023/12/21 04:59:13 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2023/12/21 04:59:23 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2023/12/21 04:59:30 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/home/pytholic/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/sklearn/svm/_c